In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization,LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.python.data import Dataset, AUTOTUNE

In [2]:
from tensorflow.python.eager.context import PhysicalDevice
from typing import List
print("TensorFlow version:", tf.__version__)
# setting the growth GPU memory occupying and print devices
try:
    devices: List[PhysicalDevice] = tf.config.list_physical_devices('GPU')
    for device in devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("Use devices:", list(map(lambda d: d.name, devices)))
except IndexError:
    print("Use CPU")

TensorFlow version: 2.6.0
Use devices: ['/physical_device:GPU:0']


# load Data

In [3]:
cifar100 = tf.keras.datasets.cifar100
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

In [4]:
pair_dataset = Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = Dataset.from_tensor_slices((test_images, test_labels))
# zip images and labels as pairs

# split the train and validation dataset
val_size = int(len(train_images)* 0.2)
train_dataset = pair_dataset.skip(val_size)
val_dataset = pair_dataset.take(val_size)

# define the batch_size and shuffle
train_dataset = train_dataset.cache().shuffle(buffer_size=1000).batch(256).prefetch(AUTOTUNE)
val_dataset = val_dataset.cache().batch(64).prefetch(AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(buffer_size=1000).batch(256).prefetch(AUTOTUNE)

# Task 1 Baseline CNN model

In [5]:
Baseline_CNN_model = Sequential([
    Conv2D(16,7,strides= 1,padding='valid', activation="relu", input_shape=(32, 32, 3)),
    MaxPooling2D(2, strides=2),
    Conv2D(32, 5,strides= 1,padding='valid', activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(100),
])

# TODO - Print summary
Baseline_CNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        2368      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 32)          12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1

In [6]:
learning_rate = 1e-3
lr_decay = ExponentialDecay(learning_rate, decay_steps=100, decay_rate=0.96)

In [7]:
Baseline_CNN_model.compile(optimizer=Adam(lr_decay),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
history = Baseline_CNN_model.fit(train_dataset, validation_data=val_dataset, epochs=50)

Epoch 1/50
157/157 [==============================] - 27s 7ms/step - loss: 5.6440 - accuracy: 0.0154 - val_loss: 4.4861 - val_accuracy: 0.0270
Epoch 2/50
157/157 [==============================] - 1s 6ms/step - loss: 4.4249 - accuracy: 0.0345 - val_loss: 4.3756 - val_accuracy: 0.0384
Epoch 3/50
157/157 [==============================] - 1s 6ms/step - loss: 4.3112 - accuracy: 0.0500 - val_loss: 4.3619 - val_accuracy: 0.0421
Epoch 4/50
157/157 [==============================] - 1s 6ms/step - loss: 4.2281 - accuracy: 0.0655 - val_loss: 4.2033 - val_accuracy: 0.0666
Epoch 5/50
157/157 [==============================] - 1s 6ms/step - loss: 4.1223 - accuracy: 0.0786 - val_loss: 4.1184 - val_accuracy: 0.0817
Epoch 6/50
157/157 [==============================] - 1s 6ms/step - loss: 4.0135 - accuracy: 0.0942 - val_loss: 4.0447 - val_accuracy: 0.0875
Epoch 7/50
157/157 [==============================] - 1s 6ms/step - loss: 3.9184 - accuracy: 0.1083 - val_loss: 3.9924 - val_accuracy: 0.0977
Epoch

In [8]:
loss, acc = Baseline_CNN_model.evaluate(test_dataset)
print("Accuracy:", acc)

40/40 [==============================] - 0s 2ms/step - loss: 3.6444 - accuracy: 0.1958
Accuracy: 0.19580000638961792


# Task 2 Best model

Using Rescaling, Data Normalization, Data augmentation ,Batch Normalizationand,layer normalization and ResNet18 model to get best accuracy

In [9]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling,RandomFlip, RandomRotation, RandomZoom
data_preprocess = Sequential([
  Rescaling(1 / 255,input_shape=(32,32,3)),
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
  RandomZoom(0.1)
])

In [10]:
# TODO - Define ResNet18 model.
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, BatchNormalization, ReLU, Add, GlobalAvgPool2D, Input,LayerNormalization

from enum import Enum


class ResidualBlockType(Enum):
    SHALLOW = "shallow"  # the block used in ResNet18 and ResNet34
    DEEP = "deep"  # the block used in ResNet50, ResNet101 and ResNet152


class ResidualBlock(Layer):
    class ShallowPath(Layer):
        def __init__(self, filters: int, strides: int):
            super().__init__()
            self.conv0 = Conv2D(filters, kernel_size=3, padding="same", strides=strides)
            self.bn0 = BatchNormalization()
            self.relu0 = ReLU()
            self.layer_norm0 = LayerNormalization(axis=3 , center=True , scale=True)
            self.conv1 = Conv2D(filters, kernel_size=3, padding="same")
            self.bn1 = BatchNormalization()
            self.layer_norm1 = LayerNormalization(axis=3 , center=True , scale=True)
        def call(self, inputs, *args, **kwargs):
            x = self.conv0(inputs)
            x = self.layer_norm0(x)
            x = self.bn0(x)
            x = self.relu0(x)

            x = self.conv1(x)
            x = self.layer_norm1(x)
            x = self.bn1(x)

            return x

    class DeepPath(Layer):
        def __init__(self, filters: int, strides: int):
            super().__init__()
            self.conv0 = Conv2D(filters, kernel_size=1, padding="same", strides=strides)
            self.bn0 = BatchNormalization()
            self.layer_norm0 = LayerNormalization(axis=3 , center=True , scale=True)
            self.relu0 = ReLU()

            self.conv1 = Conv2D(filters, kernel_size=3, padding="same")
            self.bn1 = BatchNormalization()
            self.layer_norm1 = LayerNormalization(axis=3 , center=True , scale=True)
            self.relu1 = ReLU()

            self.conv2 = Conv2D(filters * 4, kernel_size=1, padding="same")
            self.bn2 = BatchNormalization()
            self.layer_norm2 = LayerNormalization(axis=3 , center=True , scale=True)

        def call(self, inputs, *args, **kwargs):
            x = self.conv0(inputs)
            x = self.layer_norm0(x)
            x = self.bn0(x)
            x = self.relu0(x)

            x = self.conv1(x)
            x = self.layer_norm1(x)
            x = self.bn1(x)
            x = self.relu1(x)

            x = self.conv2(x)
            x = self.layer_norm2(x)
            x = self.bn2(x)

            return x

    def __init__(self, filters: int, strides: int, repeat: int,
                 architecture: ResidualBlockType = ResidualBlockType.SHALLOW):
        super().__init__()
        if architecture == ResidualBlockType.SHALLOW:
            MainPath = ResidualBlock.ShallowPath
        elif architecture == ResidualBlockType.DEEP:
            MainPath = ResidualBlock.DeepPath
        else:
            raise ValueError("Unknown residual block type")

        self.repeat = repeat
        self.blocks = []
        for i in range(repeat):
            main_path = MainPath(filters, strides=strides) if i == 0 else MainPath(filters, 1)
            residual_path = Conv2D(filters, kernel_size=1,
                                   strides=strides) if i == 0 else Layer()  # Layer class used here as Identity layer
            addition = Add()
            relu = ReLU()
            self.blocks.append((main_path, residual_path, addition, relu))

    def call(self, inputs, *args, **kwargs):
        x = inputs
        for i in range(self.repeat):
            main_path, residual_path, addition, relu = self.blocks[i]
            main_path_x = main_path(x)
            residual_path_x = residual_path(x)
            x = addition([main_path_x, residual_path_x])
            x = relu(x)
        return x


class ResNet18(Model):

    def __init__(self):
        super().__init__()
        self.rescaling = tf.keras.layers.Rescaling(1/255)
        self.norm_data = tf.keras.layers.Normalization()
        self.conv1 = Conv2D(64, 7, strides=2, padding="same")
        self.bn1 = BatchNormalization()
        self.layer_norm = LayerNormalization(axis=3 , center=True , scale=True)
        self.relu1 = ReLU()
        self.pool1 = MaxPooling2D(3, strides=2, padding="same")

        residual_type = ResidualBlockType.SHALLOW

        self.conv2_x = ResidualBlock(filters=64, strides=1, repeat=2, architecture=residual_type)
        self.conv3_x = ResidualBlock(filters=128, strides=2, repeat=2, architecture=residual_type)
        self.conv4_x = ResidualBlock(filters=256, strides=2, repeat=2, architecture=residual_type)
        self.conv5_x = ResidualBlock(filters=512, strides=2, repeat=2, architecture=residual_type)

        self.global_pool = GlobalAvgPool2D()
        self.fc = Dense(100)

        self.build(input_shape=(None, 32, 32, 3))
        self.call(Input(shape=(32, 32, 3)))

    def call(self, inputs, training=None, mask=None):
        x = self.rescaling(inputs)
        x = self.norm_data(x)
        x = self.conv1(x)
        x = self.layer_norm(x)
        x = self.bn1(x)

        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.global_pool(x)
        x = self.fc(x)

        return x

    def get_config(self):
        pass
best_model = ResNet18()
best_model.summary()

Model: "res_net18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 32, 32, 3)         0         
_________________________________________________________________
normalization (Normalization (None, 32, 32, 3)         7         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        9472      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 64)        256       
_________________________________________________________________
layer_normalization (LayerNo (None, 16, 16, 64)        128       
_________________________________________________________________
re_lu (ReLU)                 (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0 

In [12]:
best_model.compile(optimizer=Adam(lr_decay),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
epochs = 50
history_18 = best_model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)
loss_1, acc_1 = best_model.evaluate(test_dataset)
print("Accuracy:", acc_1)

Epoch 1/50
157/157 [==============================] - 13s 66ms/step - loss: 1.8567 - accuracy: 0.4904 - val_loss: 3.0054 - val_accuracy: 0.2841
Epoch 2/50
157/157 [==============================] - 10s 61ms/step - loss: 1.5424 - accuracy: 0.5664 - val_loss: 3.3696 - val_accuracy: 0.2568
Epoch 3/50
157/157 [==============================] - 10s 62ms/step - loss: 1.2960 - accuracy: 0.6244 - val_loss: 3.1897 - val_accuracy: 0.2990
Epoch 4/50
157/157 [==============================] - 10s 61ms/step - loss: 1.0613 - accuracy: 0.6891 - val_loss: 3.2840 - val_accuracy: 0.3032
Epoch 5/50
157/157 [==============================] - 10s 61ms/step - loss: 0.8483 - accuracy: 0.7473 - val_loss: 3.7727 - val_accuracy: 0.2745
Epoch 6/50
157/157 [==============================] - 10s 61ms/step - loss: 0.6508 - accuracy: 0.8058 - val_loss: 3.6311 - val_accuracy: 0.2955
Epoch 7/50
157/157 [==============================] - 10s 62ms/step - loss: 0.5172 - accuracy: 0.8434 - val_loss: 3.8138 - val_accuracy:

# Task 3
add attention layer to the resNet18

In [13]:
!pip install tensorflow-addons

     -------------------------------------- 755.7/755.7 KB 6.8 MB/s eta 0:00:00


In [15]:
# TODO - Define ResNet18 model.
import tensorflow_addons as tfa

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, BatchNormalization, ReLU, Add, GlobalAvgPool2D, Input,LayerNormalization,Attention,Softmax
# from tensorflow.keras.layers import Attention,Reshape,Input,Conv2D,Conv2DTranspose
from tensorflow.keras.models import Model

from enum import Enum


class ResidualBlockType(Enum):
    SHALLOW = "shallow"  # the block used in ResNet18 and ResNet34
    DEEP = "deep"  # the block used in ResNet50, ResNet101 and ResNet152


class ResidualBlock(Layer):
    class ShallowPath(Layer):
        def __init__(self, filters: int, strides: int):
            super().__init__()
            self.conv0 = Conv2D(filters, kernel_size=3, padding="same", strides=strides)
            self.bn0 = BatchNormalization()
            self.relu0 = ReLU()
            self.layer_norm0 = LayerNormalization(axis=3 , center=True , scale=True)
            self.conv1 = Conv2D(filters, kernel_size=3, padding="same")
            self.bn1 = BatchNormalization()
            self.layer_norm1 = LayerNormalization(axis=3 , center=True , scale=True)
        def call(self, inputs, *args, **kwargs):
            x = self.conv0(inputs)
            x = self.layer_norm0(x)
            x = self.bn0(x)
            x = self.relu0(x)

            x = self.conv1(x)
            x = self.layer_norm1(x)
            x = self.bn1(x)

            return x

    class DeepPath(Layer):
        def __init__(self, filters: int, strides: int):
            super().__init__()
            self.conv0 = Conv2D(filters, kernel_size=1, padding="same", strides=strides)
            self.bn0 = BatchNormalization()
            self.layer_norm0 = LayerNormalization(axis=3 , center=True , scale=True)
            self.relu0 = ReLU()

            self.conv1 = Conv2D(filters, kernel_size=3, padding="same")
            self.bn1 = BatchNormalization()
            self.layer_norm1 = LayerNormalization(axis=3 , center=True , scale=True)
            self.relu1 = ReLU()

            self.conv2 = Conv2D(filters * 4, kernel_size=1, padding="same")
            self.bn2 = BatchNormalization()
            self.layer_norm2 = LayerNormalization(axis=3 , center=True , scale=True)

        def call(self, inputs, *args, **kwargs):
            x = self.conv0(inputs)
            x = self.layer_norm0(x)
            x = self.bn0(x)
            x = self.relu0(x)

            x = self.conv1(x)
            x = self.layer_norm1(x)
            x = self.bn1(x)
            x = self.relu1(x)

            x = self.conv2(x)
            x = self.layer_norm2(x)
            x = self.bn2(x)

            return x

    def __init__(self, filters: int, strides: int, repeat: int,
                 architecture: ResidualBlockType = ResidualBlockType.SHALLOW):
        super().__init__()
        if architecture == ResidualBlockType.SHALLOW:
            MainPath = ResidualBlock.ShallowPath
        elif architecture == ResidualBlockType.DEEP:
            MainPath = ResidualBlock.DeepPath
        else:
            raise ValueError("Unknown residual block type")

        self.repeat = repeat
        self.blocks = []
        for i in range(repeat):
            main_path = MainPath(filters, strides=strides) if i == 0 else MainPath(filters, 1)
            residual_path = Conv2D(filters, kernel_size=1,
                                   strides=strides) if i == 0 else Layer()  # Layer class used here as Identity layer
            addition = Add()
            relu = ReLU()
            self.blocks.append((main_path, residual_path, addition, relu))

    def call(self, inputs, *args, **kwargs):
        x = inputs
        for i in range(self.repeat):
            main_path, residual_path, addition, relu = self.blocks[i]
            main_path_x = main_path(x)
            residual_path_x = residual_path(x)
            x = addition([main_path_x, residual_path_x])
            x = relu(x)
        return x


class ResNet18_attention(Model):

    def __init__(self):
        super().__init__()
        self.rescaling = tf.keras.layers.Rescaling(1/255)
        self.norm_data = tf.keras.layers.Normalization()
        self.conv1 = Conv2D(64, 7, strides=2, padding="same")
        self.attention = Sequential([Conv2D(64,7, strides=2, padding="same"),tfa.layers.AdaptiveAveragePooling2D((8,8)),Softmax()])
        self.bn1 = BatchNormalization()
        self.layer_norm = LayerNormalization(axis=3 , center=True , scale=True)
        self.relu1 = ReLU()
        self.pool1 = MaxPooling2D(3, strides=2, padding="same")

        residual_type = ResidualBlockType.SHALLOW

        self.conv2_x = ResidualBlock(filters=64, strides=1, repeat=2, architecture=residual_type)
        self.conv3_x = ResidualBlock(filters=128, strides=2, repeat=2, architecture=residual_type)
        self.conv4_x = ResidualBlock(filters=256, strides=2, repeat=2, architecture=residual_type)
        self.conv5_x = ResidualBlock(filters=512, strides=2, repeat=2, architecture=residual_type)

        self.global_pool = GlobalAvgPool2D()
        self.fc = Dense(100)

        self.build(input_shape=(None, 32, 32, 3))
        self.call(Input(shape=(32, 32, 3)))

    def call(self, inputs, training=None, mask=None):
        x = self.rescaling(inputs)
        x = self.norm_data(x)
        x = self.conv1(x)

        x_attention =  self.attention(x)
        x = self.layer_norm(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = x*x_attention
        x = self.conv2_x(x)

        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.global_pool(x)
        x = self.fc(x)

        return x

    def get_config(self):
        pass
task_3 = ResNet18_attention()
task_3 .summary()

Model: "res_net18_attention_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 32, 32, 3)         0         
_________________________________________________________________
normalization_2 (Normalizati (None, 32, 32, 3)         7         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 16, 16, 64)        9472      
_________________________________________________________________
sequential_3 (Sequential)    (None, 8, 8, 64)          200768    
_________________________________________________________________
batch_normalization_34 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
layer_normalization_34 (Laye (None, 16, 16, 64)        128       
_________________________________________________________________
re_lu_34 (ReLU)              (None, 16, 16, 6

In [16]:
learning_rate = 1e-3
lr_decay = ExponentialDecay(learning_rate, decay_steps=100, decay_rate=0.96)
task_3.compile(optimizer=Adam(lr_decay),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
epochs = 50
history_18 = task_3.fit(train_dataset, validation_data=val_dataset, epochs=epochs)
loss_1, acc_1 = task_3.evaluate(test_dataset)
print("Accuracy:", acc_1)

Epoch 1/50
157/157 [==============================] - 15s 75ms/step - loss: 3.7093 - accuracy: 0.1499 - val_loss: 4.4633 - val_accuracy: 0.0471
Epoch 2/50
157/157 [==============================] - 11s 71ms/step - loss: 3.0237 - accuracy: 0.2503 - val_loss: 3.9909 - val_accuracy: 0.1008
Epoch 3/50
157/157 [==============================] - 11s 71ms/step - loss: 2.6428 - accuracy: 0.3214 - val_loss: 3.2032 - val_accuracy: 0.2252
Epoch 4/50
157/157 [==============================] - 11s 71ms/step - loss: 2.3446 - accuracy: 0.3830 - val_loss: 3.8718 - val_accuracy: 0.1800
Epoch 5/50
157/157 [==============================] - 11s 71ms/step - loss: 2.1012 - accuracy: 0.4336 - val_loss: 3.8378 - val_accuracy: 0.1937
Epoch 6/50
157/157 [==============================] - 11s 71ms/step - loss: 1.8710 - accuracy: 0.4853 - val_loss: 3.5909 - val_accuracy: 0.2142
Epoch 7/50
157/157 [==============================] - 11s 71ms/step - loss: 1.6458 - accuracy: 0.5391 - val_loss: 3.5619 - val_accuracy: